In [2]:
import sys
# Set the path to the repository here
print(sys.path)
sys.path.append("/home/falcon2212/college/sem8/PE/detr-tensorflow/")
import detr_tf

['/home/falcon2212/college/sem8/PE/detr-tensorflow/environments', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/home/falcon2212/college/sem8/PE/detr-tensorflow/environments/facemaskdetector_env/lib/python3.7/site-packages', '/home/falcon2212/college/sem8/PE/detr-tensorflow/environments/facemaskdetector_env/lib/python3.7/site-packages/IPython/extensions', '/home/falcon2212/.ipython']


In [3]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 1:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
from detr_tf.training_config import TrainingConfig
from os.path import expanduser
import os

class CustomConfig(TrainingConfig):

    def __init__(self):
        super().__init__()        
        # Dataset info
        self.datadir = os.path.join(expanduser("/home/falcon2212/college/sem8/PE"), "data/hardhat/")
        # The model is trained using fixed size images.
        # The following is the desired target image size, but it can be change based on your
        # dataset
        self.image_size = (480, 720)
        # Batch size
        self.batch_size = 1
        # Using the target batch size , the training loop will agregate the gradient on 38 steps
        # before to update the weights
        self.target_batch = 8

config = CustomConfig()

In [6]:
from detr_tf.data import load_tfcsv_dataset

# Load the dataset and exclude the person class (for some reason not all person are labeled on the training set)
train_iterator, class_names = load_tfcsv_dataset("train", config.batch_size, config, augmentation=True, exclude=["person"])
valid_iterator, class_names = load_tfcsv_dataset("test", config.batch_size, config, augmentation=False, exclude=["person"])
print("class_names", class_names)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from detr_tf.networks.detr import get_detr_model
# Load the pretrained DETR model with new heads at the top
# include_top: We do not include the last layers that predicts the bbox pos and class (include_top=False)
# nb_class: We add new layers on top of the model to predicts the bbox pos and class with three class (nb_class=3), background, helmet, face
# weights: Use the "detr" weight to init the model
detr = get_detr_model(config, include_top=False, nb_class=3, weights="detr")
detr.summary()

In [ ]:
# Train/finetune the transformers only
config.train_backbone = tf.Variable(False)
config.train_transformers = tf.Variable(False)
config.train_nlayers = tf.Variable(True)

In [ ]:
config.nlayers_lr = tf.Variable(1e-3)

In [ ]:
from detr_tf.optimizers import setup_optimizers
# Setup the optimziers and the trainable variables
optimzers = setup_optimizers(detr, config)

In [ ]:
from detr_tf import training
training.fit(detr, train_iterator, optimzers, config, epoch_nb=0, class_names=class_names)